## Item 기반 협업 필터링

***



In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [2]:
ratings = pd.read_csv('ratings.csv')

In [3]:
movies = pd.read_csv('movies.csv')

In [4]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,307,3.5,1256677221
1,1,481,3.5,1256677456
2,1,1091,1.5,1256677471
3,1,1257,4.5,1256677460
4,1,1449,4.5,1256677264


In [6]:
movies.shape, ratings.shape

((58098, 3), (27753444, 4))

In [8]:
ratings.drop('timestamp', axis = 1, inplace = True)

timestamp 변수 삭제

In [10]:
data = pd.merge(ratings, movies, on = 'movieId', how = 'left')

In [20]:
over200 = data.groupby('userId').movieId.size()[data.groupby('userId').movieId.size() > 200].index

200편 이상 영화를 본 유저 정보만 활용

In [24]:
data = data.query('userId in @over200')

In [39]:
u_m = pd.crosstab(data.movieId, data.userId, values = data.rating, aggfunc = lambda x : x)

각 영화에 대한 개별 유저들이 평가한 rating을 값으로 채워넣는다.

In [40]:
u_m.fillna(0, inplace = True)

안본영화에 대해서는 당연히 rating이 없기 때문에 0으로 채워넣는다.

In [75]:
u_m

userId,4,19,42,43,51,55,56,71,73,79,...,283153,283164,283165,283170,283183,283184,283185,283195,283204,283224
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,5.0,4.0,5.0,2.5,4.0,4.0,5.0,...,3.0,0.0,2.5,0.0,5.0,4.0,0.0,5.0,0.0,0.0
2,4.0,0.0,3.0,0.0,3.0,0.0,3.5,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,2.5,0.0,4.0,0.0,0.0
3,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,4.5,0.0,4.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0
5,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193876,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
193878,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
193880,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
from sklearn.metrics.pairwise import cosine_similarity

In [41]:
item_based = cosine_similarity(u_m)

In [42]:
item_based_result = pd.DataFrame(item_based, index = u_m.index, columns = u_m.index)

In [44]:
item_based_result.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193861,193863,193864,193866,193868,193876,193878,193880,193882,193886
movieId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.605636,0.351004,0.167633,0.349167,0.504736,0.354755,0.147217,0.178504,0.558716,...,0.0,0.007282,0.004854,0.008091,0.009608,0.004045,0.007282,0.002427,0.000000,0.0
2,0.605636,1.000000,0.348899,0.161001,0.351262,0.387990,0.321710,0.193392,0.203444,0.507094,...,0.0,0.000000,0.000000,0.006478,0.012911,0.007773,0.000000,0.006478,0.000000,0.0
3,0.351004,0.348899,1.000000,0.229844,0.427669,0.292589,0.366821,0.182812,0.247787,0.346289,...,0.0,0.000000,0.000000,0.015830,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
4,0.167633,0.161001,0.229844,1.000000,0.256455,0.146665,0.269129,0.151115,0.164504,0.143056,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.004617,0.0
5,0.349167,0.351262,0.427669,0.256455,1.000000,0.243714,0.393035,0.212279,0.217599,0.314273,...,0.0,0.000000,0.000000,0.000000,0.009595,0.000000,0.000000,0.000000,0.000000,0.0


In [67]:
def get_rec_movie(idx) :
    movie_idx = item_based_result[idx].sort_values(ascending = False)[1:11].index
    movie_list = movies.query('movieId in @movie_idx')[['title', 'genres']]
    target_movie = movies[movies.movieId == idx].title.values[0]
    print(f'{target_movie}를 시청한 유저에게 추천할 영화 Top 10')
    return movie_list

In [68]:
get_rec_movie(3)

Grumpier Old Men (1995)를 시청한 유저에게 추천할 영화 Top 10


,title,genres
4,Father of the Bride Part II (1995),Comedy
364,Maverick (1994),Adventure|Comedy|Western
428,City Slickers II: The Legend of Curly's Gold (...,Adventure|Comedy|Western
496,Mrs. Doubtfire (1993),Comedy|Drama
535,Sleepless in Seattle (1993),Comedy|Drama|Romance
581,Ghost (1990),Comedy|Drama|Fantasy|Romance|Thriller
591,Pretty Woman (1990),Comedy|Romance
724,Twister (1996),Action|Adventure|Romance|Thriller
775,"Nutty Professor, The (1996)",Comedy|Fantasy|Romance|Sci-Fi
3362,Grumpy Old Men (1993),Comedy


In [73]:
get_rec_movie(1500)

Grosse Pointe Blank (1997)를 시청한 유저에게 추천할 영화 Top 10


,title,genres
49,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
220,Clerks (1994),Comedy
602,Fargo (1996),Comedy|Crime|Drama|Thriller
1172,"Princess Bride, The (1987)",Action|Adventure|Comedy|Fantasy|Romance
1237,Groundhog Day (1993),Comedy|Fantasy|Romance
1564,L.A. Confidential (1997),Crime|Film-Noir|Mystery|Thriller
1582,Chasing Amy (1997),Comedy|Drama|Romance
1885,"Breakfast Club, The (1985)",Comedy|Drama
2833,Ferris Bueller's Day Off (1986),Comedy
3393,High Fidelity (2000),Comedy|Drama|Romance
